In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
root_path = 'gdrive/My Drive/colab_notebooks/'

In [4]:
import random
import numpy as np
import keras

from keras.datasets import cifar10
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image

import sys
import pandas as pd
import pickle
import os
%matplotlib inline

import cv2
import time
import imageio

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255

170500096/170498071 [==============================] - 3s 0us/step


In [ ]:
train_folder = "gdrive/My Drive/colab_notebooks/data/train/"
test_folder = "gdrive/My Drive/colab_notebooks/data/test/"
save_path = "gdrive/My Drive/colab_notebooks/data/prepared/"

In [7]:
automobiles = 0
dogs = 0
horses = 0
for i in range (len(y_train)):
    if y_train[i] == 1:
#         data = np.random.random((100,100))

        #Rescale to 0-255 and convert to uint8
#         rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
        if automobiles < 100:
          im = Image.fromarray(x_train[i])
          im.save('gdrive/My Drive/colab_notebooks/data_test/automobile'+ str(i) +'.png')
          automobiles += 1
    if y_train[i] == 5:
#         data = np.random.random((100,100))

        #Rescale to 0-255 and convert to uint8
#         rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
        if dogs < 100:
          im = Image.fromarray(x_train[i])
          im.save('gdrive/My Drive/colab_notebooks/data_test/dog'+ str(i) +'.png')
          dogs += 1
    if y_train[i] == 7:
#         data = np.random.random((100,100))

        #Rescale to 0-255 and convert to uint8
#         rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)
        if horses < 100:
          im = Image.fromarray(x_train[i])
          im.save('gdrive/My Drive/colab_notebooks/data_test/horse'+ str(i) +'.png')
          horses += 1
    if (dogs == 100) and (automobiles == 100) and (horses == 100):
      break

In [ ]:
for i in range (len(y_test)):
    if y_test[i] == 1:
#         data = np.random.random((100,100))

        #Rescale to 0-255 and convert to uint8
#         rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)

        im = Image.fromarray(x_test[i])
        im.save('gdrive/My Drive/colab_notebooks/data_test/automobile'+ str(i) +'.png')
    if y_test[i] == 5:
#         data = np.random.random((100,100))

        #Rescale to 0-255 and convert to uint8
#         rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)

        im = Image.fromarray(x_test[i])
        im.save('gdrive/My Drive/colab_notebooks/data_test/dog'+ str(i) +'.png')
    if y_test[i] == 7:
#         data = np.random.random((100,100))

        #Rescale to 0-255 and convert to uint8
#         rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)

        im = Image.fromarray(x_test[i])
        im.save('gdrive/My Drive/colab_notebooks/data_test/horse'+ str(i) +'.png')

In [ ]:
def loadimgs(path, batch_size,n = 0):
    '''
    path => Path of train directory or test directory
    '''
    X=[]
    y = []
    cat_dict = {}
    category_dict = {}
    curr_y = n
    
    # we load every class seperately so we can isolate them later
    for category in os.listdir(path):
        print("loading cateogry: " + category)
        category_dict[category] = [curr_y,None]
        category_path = os.path.join(path,category)
        category_images=[]
        # read all the images in the current category
        for filename in os.listdir(category_path):
            if(len(category_images) == batch_size):
                break
            image_path = os.path.join(category_path, filename)
            image = imageio.imread(image_path)
            category_images.append(image)
            y.append(curr_y)
            try:
                X.append(np.stack(category_images))
            # edge case  - last one
            except ValueError as e:
                print(e)
                print("error - category_images:", category_images)
            curr_y += 1
            category_dict[category][1] = curr_y - 1
            
    y = np.vstack(y)
    X = np.stack(X)
    return X,y,category_dict

In [ ]:
X,y,c = loadimgs(train_folder, 32)

# pairs_train, labels_train = make_pairs(x_train, y_train)
# pairs_test, labels_test = make_pairs(x_test, y_test)

loading cateogry: automobile
loading cateogry: dog
loading cateogry: horse


ValueError: ignored

In [ ]:
X.shape
y.shape
c.keys()

In [ ]:
def get_batch(batch_size,s="train"):
    """
    Create batch of n pairs, half same class, half different class
    """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h = X.shape
    
    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
    
    return pairs, targets

In [ ]:
def generate(batch_size, s="train"):
    """
    a generator for batches, so model.fit_generator can be used.
    """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [ ]:
def get_siamese_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [ ]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)



In [ ]:
from google.colab import files
os.listdir('./')
# files.download("model.hfs5")

['.config', 'gdrive', 'sample_data']